In [9]:
import cv2
import mediapipe as mp
from numpy import argmax
from AppOpener import run
from tensorflow.keras.models import load_model
from screen_brightness_control import set_brightness

In [10]:
model = load_model(r'C:\Users\Development\Desktop\IEEE\Model\model.h5')

### Class for hand detection

In [11]:
class Detector :
    def __init__(self) :
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode=False, max_num_hands=1,
                                        min_detection_confidence=0.5,
                                        min_tracking_confidence=0.5)
        
    # Function to return coordinates of detected hand, if any
    def findHands(self, img, draw=True, flipType=True):

            res = []

            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = self.hands.process(imgRGB)
            allHands = []
            h, w, c = img.shape
            if results.multi_hand_landmarks:
                for handType, handLms in zip(results.multi_handedness, results.multi_hand_landmarks):

                    # Landmark list
                    mylmList = []
                    xList = []
                    yList = []
                    for id, lm in enumerate(handLms.landmark):
                        px, py, pz = int(lm.x * w), int(lm.y * h), int(lm.z * w)
                        mylmList.append([px, py, pz])
                        xList.append(px)
                        yList.append(py)

                    # Bounding box
                    xmin, xmax = min(xList), max(xList)
                    ymin, ymax = min(yList), max(yList)
                    boxW, boxH = xmax - xmin, ymax - ymin
                    bbox = xmin, ymin, boxW, boxH

                    if draw:
                        res.append(bbox)

            if draw:
                return res

### Performing OS operation based on gesture

In [12]:
cap = cv2.VideoCapture(0)

detector = Detector()
while True:
    ret, image = cap.read()
    image = cv2.flip(image, 1)
    hand_coor = detector.findHands(image)
    
    try :
        if hand_coor:
            for (x,y,w,h) in hand_coor:
                cv2.rectangle(image,(x-20,y-20),(x+w+20,y+h+20),(0,255,255),2)
                cropped_hand = image[y-20:y+h+20, x-20:x+w+20]
                cropped_hand = cv2.resize(cropped_hand, (224,224))

                ci = cv2.resize(cropped_hand,(224,224), interpolation = cv2.INTER_LINEAR)
                ci = cropped_hand / 255.
                ci = cropped_hand.reshape(1,224,224,3) 
                res = argmax(model.predict(ci), axis=1)

                text = ""
                if res[0] == 0:             # Closed palm
                    set_brightness(25)
                    text = "Decreasing Brightness..."
                elif res[0] == 1:           # 5-finger
                    set_brightness(100)
                    text = "Increasing Brightness..."
                elif res[0] == 2:           # 2-finger
                    run("media player")
                    text = "Launching Video Player..."
                elif res[0] == 3:           # 3-finger
                    run("word")
                    text = "Launching Word..."
                    
                cv2.putText(image, text, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                    
    except:
            print("No hand detected!")
            
    exit = cv2.waitKey(1)
    
    if exit == 13:
        break
    cv2.imshow("Hand Gesture Recognition", image)
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 171ms/step
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
1/1 [==============================] - 0s 176ms/step
OPENING MEDIA PLAYER
1/1 [==============================] - 0s 173ms/step
OPENING MEDIA PLAYER
1/1 [==============================] - 0s 175ms/step
OPENING MEDIA PLAYER
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
1/1 [==============================] - 0s 236ms/step
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
No hand detected!
1/1 [==============================] - 0s 